In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate as cv
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
from surprise.model_selection import cross_validate
import random
from datetime import datetime
from sklearn.model_selection import KFold
from collections import defaultdict

import pandas as pd
from rake_nltk import Rake
#pip install rake-nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
#Para garantizar reproducibilidad en resultados
seed = 10
random.seed(seed)
np.random.seed(seed)

# importing the module
import imdb
   
# creating instance of IMDb
ia = imdb.IMDb()

In [204]:
def get_top5_recommendations(predictions, topN = 5):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

In [2]:
ratings=pd.read_csv('ml-latest-small/ratings.csv',sep=',')
movies= pd.read_csv('ml-latest-small/movies.csv')
links= pd.read_csv('ml-latest-small/links.csv')

ratings['timestamp'] = ratings['timestamp'].apply(datetime.fromtimestamp)
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-30 13:45:03
1,1,3,4.0,2000-07-30 13:20:47
2,1,6,4.0,2000-07-30 13:37:04
3,1,47,5.0,2000-07-30 14:03:35
4,1,50,5.0,2000-07-30 13:48:51
...,...,...,...,...
100831,610,166534,4.0,2017-05-03 16:53:22
100832,610,168248,5.0,2017-05-03 17:21:31
100833,610,168250,5.0,2017-05-08 14:50:47
100834,610,168252,5.0,2017-05-03 16:19:12


In [13]:
movie_links=movies.merge(links,on = 'movieId',how = 'inner')
movie_links.head(3)
movie_links['directors']= None
movie_links['actors']= None
movie_links

,movieId,title,genres,imdbId,tmdbId,directors,actors
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,None,None
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,None,None
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,None,None
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,None,None
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,None,None
...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,5476944,432131.0,None,None
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,5914996,445030.0,None,None
9739,193585,Flint (2017),Drama,6397426,479308.0,None,None
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,8391976,483455.0,None,None


In [14]:
movie_links = movie_links.sample(frac=0.05)
movie_links.describe

<bound method NDFrame.describe of       movieId                                          title  \
2535     3395                                  Nadine (1987)   
5024     7812                   Babylon 5: Thirdspace (1998)   
6359    49647                         Charlotte's Web (2006)   
3289     4449                              Adanggaman (2000)   
3440     4688                              Black Robe (1991)   
...       ...                                            ...   
2814     3761                     Blood In, Blood Out (1993)   
9453   167772                 The Spirit of Christmas (1995)   
5908    33669  Sisterhood of the Traveling Pants, The (2005)   
5353     8920                       Country Girl, The (1954)   
2430     3241                       Cup, The (Phörpa) (1999)   

                             genres  imdbId   tmdbId directors actors  
2535                         Comedy   93596  61178.0      None   None  
5024                   Drama|Sci-Fi  121804  10941.0 

In [15]:
i=0
for i in movie_links.index:
    try:
        movie_data = ia.get_movie(str(movie_links['imdbId'][i]))
        #print(movie_data['directors'])
        directors = []
        actors = []
        for director in movie_data['directors']:
            directors.append(director['name'])
        for director in movie_data['actors']:
            actors.append(director['name'])
        movie_links['directors'][i] = directors
        movie_links['actors'][i] = actors
        i = i+1
        if i==20:
            print('Pelicula 20')
            i=0
    except:
        print(movie_links['imdbId'][i])
        #movie_links['actors'][i] = movie_data['actors']
        #print(movie_links['imdbId'])

<ipython-input-15-93db708ad9a3>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_links['directors'][i] = directors
<ipython-input-15-93db708ad9a3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_links['actors'][i] = actors


112130
3106868
233044
91455
479113


In [16]:
movie_links

,movieId,title,genres,imdbId,tmdbId,directors,actors
2535,3395,Nadine (1987),Comedy,93596,61178.0,[Robert Benton],"[Jeff Bridges, Kim Basinger, Rip Torn, Gwen Ve..."
5024,7812,Babylon 5: Thirdspace (1998),Drama|Sci-Fi,121804,10941.0,[Jesús Salvador Treviño],"[Bruce Boxleitner, Claudia Christian, Mira Fur..."
6359,49647,Charlotte's Web (2006),Children|Comedy|Drama|Fantasy,413895,9986.0,[Gary Winick],"[Julia Roberts, Steve Buscemi, John Cleese, Op..."
3289,4449,Adanggaman (2000),Drama,262210,78248.0,[Roger Gnoan M'Bala],"[Rasmané Ouédraogo, Albertine N'Guessan, Ziabl..."
3440,4688,Black Robe (1991),Adventure|Drama,101465,41784.0,[Bruce Beresford],"[Lothaire Bluteau, Aden Young, Sandrine Holt, ..."
...,...,...,...,...,...,...,...
2814,3761,"Blood In, Blood Out (1993)",Action|Crime|Drama|Thriller,106469,9702.0,[Taylor Hackford],"[Damian Chapa, Jesse Borrego, Benjamin Bratt, ..."
9453,167772,The Spirit of Christmas (1995),Animation|Comedy,122264,16487.0,"[Trey Parker, Matt Stone]","[Trey Parker, Matt Stone]"
5908,33669,"Sisterhood of the Traveling Pants, The (2005)",Adventure|Comedy|Drama,403508,9779.0,[Ken Kwapis],"[Amber Tamblyn, Alexis Bledel, America Ferrera..."
5353,8920,"Country Girl, The (1954)",Drama,46874,2438.0,[George Seaton],"[Bing Crosby, Grace Kelly, William Holden, Ant..."


In [138]:
movie_links_copy = movie_links.copy()

In [139]:
movie_links_copy['genres'] = movie_links_copy['genres'].str.split('|')
movie_links_copy

,movieId,title,genres,imdbId,tmdbId,directors,actors
2535,3395,Nadine (1987),[Comedy],93596,61178.0,[Robert Benton],"[Jeff Bridges, Kim Basinger, Rip Torn, Gwen Ve..."
5024,7812,Babylon 5: Thirdspace (1998),"[Drama, Sci-Fi]",121804,10941.0,[Jesús Salvador Treviño],"[Bruce Boxleitner, Claudia Christian, Mira Fur..."
6359,49647,Charlotte's Web (2006),"[Children, Comedy, Drama, Fantasy]",413895,9986.0,[Gary Winick],"[Julia Roberts, Steve Buscemi, John Cleese, Op..."
3289,4449,Adanggaman (2000),[Drama],262210,78248.0,[Roger Gnoan M'Bala],"[Rasmané Ouédraogo, Albertine N'Guessan, Ziabl..."
3440,4688,Black Robe (1991),"[Adventure, Drama]",101465,41784.0,[Bruce Beresford],"[Lothaire Bluteau, Aden Young, Sandrine Holt, ..."
...,...,...,...,...,...,...,...
2814,3761,"Blood In, Blood Out (1993)","[Action, Crime, Drama, Thriller]",106469,9702.0,[Taylor Hackford],"[Damian Chapa, Jesse Borrego, Benjamin Bratt, ..."
9453,167772,The Spirit of Christmas (1995),"[Animation, Comedy]",122264,16487.0,"[Trey Parker, Matt Stone]","[Trey Parker, Matt Stone]"
5908,33669,"Sisterhood of the Traveling Pants, The (2005)","[Adventure, Comedy, Drama]",403508,9779.0,[Ken Kwapis],"[Amber Tamblyn, Alexis Bledel, America Ferrera..."
5353,8920,"Country Girl, The (1954)",[Drama],46874,2438.0,[George Seaton],"[Bing Crosby, Grace Kelly, William Holden, Ant..."


In [100]:
movie_links_copy['actors']

2535    [Jeff Bridges, Kim Basinger, Rip Torn, Gwen Ve...
5024    [Bruce Boxleitner, Claudia Christian, Mira Fur...
6359    [Julia Roberts, Steve Buscemi, John Cleese, Op...
3289    [Rasmané Ouédraogo, Albertine N'Guessan, Ziabl...
3440    [Lothaire Bluteau, Aden Young, Sandrine Holt, ...
                              ...                        
2814    [Damian Chapa, Jesse Borrego, Benjamin Bratt, ...
9453                            [Trey Parker, Matt Stone]
5908    [Amber Tamblyn, Alexis Bledel, America Ferrera...
5353    [Bing Crosby, Grace Kelly, William Holden, Ant...
2430    [Orgyen Tobgyal, Neten Chokling, Jamyang Lodro...
Name: actors, Length: 487, dtype: object

In [140]:
## change all to lowercase and appen names without space
movie_links_copy['bagofwords'] =''
for i in movie_links_copy.index:
    info = ''
    if movie_links_copy['actors'][i] != None:
        for j in movie_links_copy['actors'][i]:
            info = info + ' '+j.replace(' ', '').lower()
        #print(info)
    movie_links_copy['bagofwords'][i] = movie_links_copy['bagofwords'][i] + info
movie_links_copy.drop(columns = ['actors'], inplace = True)

for i in movie_links_copy.index:
    info = ''
    if movie_links_copy['directors'][i] != None:
        for j in movie_links_copy['directors'][i]:
            info = info + ' '+j.replace(' ', '').lower()
        #print(info)
    movie_links_copy['bagofwords'][i] = movie_links_copy['bagofwords'][i] + info

movie_links_copy.drop(columns = ['directors'], inplace = True)
for i in movie_links_copy.index:
    info = ''
    if movie_links_copy['genres'][i] != None:
        for j in movie_links_copy['genres'][i]:
            info = info + ' '+j.replace(' ', '').lower()
        #print(info)
    movie_links_copy['bagofwords'][i] = movie_links_copy['bagofwords'][i] + info
        
movie_links_copy.drop(columns = ['genres'], inplace = True)
movie_links_copy

<ipython-input-140-e2b5061e1263>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_links_copy['bagofwords'][i] = movie_links_copy['bagofwords'][i] + info
<ipython-input-140-e2b5061e1263>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_links_copy['bagofwords'][i] = movie_links_copy['bagofwords'][i] + info
<ipython-input-140-e2b5061e1263>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_links_copy['bagofwords'][i] = m

,movieId,title,imdbId,tmdbId,bagofwords
2535,3395,Nadine (1987),93596,61178.0,jeffbridges kimbasinger riptorn gwenverdon gl...
5024,7812,Babylon 5: Thirdspace (1998),121804,10941.0,bruceboxleitner claudiachristian mirafurlan r...
6359,49647,Charlotte's Web (2006),413895,9986.0,juliaroberts stevebuscemi johncleese oprahwin...
3289,4449,Adanggaman (2000),262210,78248.0,rasmanéouédraogo albertinen'guessan ziablehon...
3440,4688,Black Robe (1991),101465,41784.0,lothairebluteau adenyoung sandrineholt august...
...,...,...,...,...,...
2814,3761,"Blood In, Blood Out (1993)",106469,9702.0,damianchapa jesseborrego benjaminbratt enriqu...
9453,167772,The Spirit of Christmas (1995),122264,16487.0,treyparker mattstone treyparker mattstone ani...
5908,33669,"Sisterhood of the Traveling Pants, The (2005)",403508,9779.0,ambertamblyn alexisbledel americaferrera blak...
5353,8920,"Country Girl, The (1954)",46874,2438.0,bingcrosby gracekelly williamholden anthonyro...


In [141]:
movie_links_copy.drop(columns = ['imdbId'], inplace = True)
movie_links_copy.drop(columns = ['tmdbId'], inplace = True)


In [142]:
movie_links_copy = movie_links_copy.set_index('movieId')
movie_links_copy

,title,bagofwords
movieId,,
3395,Nadine (1987),jeffbridges kimbasinger riptorn gwenverdon gl...
7812,Babylon 5: Thirdspace (1998),bruceboxleitner claudiachristian mirafurlan r...
49647,Charlotte's Web (2006),juliaroberts stevebuscemi johncleese oprahwin...
4449,Adanggaman (2000),rasmanéouédraogo albertinen'guessan ziablehon...
4688,Black Robe (1991),lothairebluteau adenyoung sandrineholt august...
...,...,...
3761,"Blood In, Blood Out (1993)",damianchapa jesseborrego benjaminbratt enriqu...
167772,The Spirit of Christmas (1995),treyparker mattstone treyparker mattstone ani...
33669,"Sisterhood of the Traveling Pants, The (2005)",ambertamblyn alexisbledel americaferrera blak...


In [143]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(movie_links_copy['bagofwords'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim 
#Let’s have a brief look at it: all the numbers on the diagonal are 1 because, 
#of course, every movie is identical to itself. The matrix is also symmetrical because the 
#similarity between A and B is the same as the similarity between B and A.

array([[1.        , 0.        , 0.02597622, ..., 0.02360961, 0.        ,
        0.02553215],
       [0.        , 1.        , 0.02418254, ..., 0.02197935, 0.02721655,
        0.        ],
       [0.02597622, 0.02418254, 1.        , ..., 0.03189099, 0.01974496,
        0.01724394],
       ...,
       [0.02360961, 0.02197935, 0.03189099, ..., 1.        , 0.01794606,
        0.0156729 ],
       [0.        , 0.02721655, 0.01974496, ..., 0.01794606, 1.        ,
        0.        ],
       [0.02553215, 0.        , 0.01724394, ..., 0.0156729 , 0.        ,
        1.        ]])

In [281]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
df = movie_links_copy
indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):

    # initializing the empty list of recommended movies
    recommended_movies = []
    try:
        # gettin the index of the movie that matches the title
        idx = indices[indices == title].index[0]

        # creating a Series with the similarity scores in descending order
        score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

        # getting the indexes of the 10 most similar movies
        top_10_indexes = list(score_series.iloc[1:11].index)

        # populating the list with the titles of the best 10 matching movies
        for i in top_10_indexes:
            recommended_movies.append(list(df.index)[i])
    except:
        i=0
        #print("not recomend")
    return recommended_movies

In [283]:
recommendations(3395)

[136443, 168632, 95377, 171695, 168846, 182293, 104644, 7316, 95311, 167772]

In [159]:
movie_links_copy.loc[[136443]]

,title,bagofwords
movieId,,
136443,Gabriel Iglesias: Hot and Fluffy (2007),gabrieliglesias mannyrodriguez comedy


In [162]:
print(movie_links_copy.loc[[3395]].bagofwords)

movieId
3395     jeffbridges kimbasinger riptorn gwenverdon gl...
Name: bagofwords, dtype: object


## filtrado colaborativo

In [221]:
movie_data_ratings_data=movies.merge(ratings,on = 'movieId',how = 'inner')
movie_data_ratings_data.head(3)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,2000-07-30 13:45:03
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1996-11-08 01:36:02
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,2005-01-25 01:52:26


In [173]:
movie_data_ratings_data.drop(columns = ['genres'], inplace = True)
movie_data_ratings_data.drop(columns = ['title'], inplace = True)

In [174]:
movie_data_ratings_data

,movieId,userId,rating,timestamp
0,1,1,4.0,2000-07-30 13:45:03
1,1,5,4.0,1996-11-08 01:36:02
2,1,7,4.5,2005-01-25 01:52:26
3,1,15,2.5,2017-11-13 07:59:30
4,1,17,4.5,2011-05-18 00:28:03
...,...,...,...,...
100831,193581,184,4.0,2018-09-16 09:44:42
100832,193583,184,3.5,2018-09-16 09:52:25
100833,193585,184,3.5,2018-09-16 09:56:45
100834,193587,184,3.5,2018-09-16 10:00:21


In [175]:
# load the model from disk
import pickle
filename = 'filtrado_colaborativo.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [243]:
# getting the number unique users and restaurants
unique_users = movie_data_ratings_data.userId.unique().shape[0]
print(unique_users)
unique_restaurants = movie_data_ratings_data.movieId.unique().shape[0]
print(unique_restaurants)
#Create two user-item matrices, one for training and another for testing
unique_users
matrix_all = np.zeros((unique_users, unique_restaurants))
#matrix_all
current_reviews= movie_data_ratings_data[ ["userId","movieId","rating"] ]
current_reviews

610
9724


,userId,movieId,rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5
...,...,...,...
100831,184,193581,4.0
100832,184,193583,3.5
100833,184,193585,3.5
100834,184,193587,3.5


In [244]:
current_reviews= movie_data_ratings_data[ ["userId","movieId","rating"] ]
current_reviews['user_id_simple'] = pd.factorize(current_reviews.userId)[0]
current_reviews['movie_id_simple'] = pd.factorize(current_reviews.movieId)[0]
for i in current_reviews.index:
    #print(current_reviews['userId'][i])
    matrix_all[current_reviews['user_id_simple'][i], current_reviews['movie_id_simple'][i]] = current_reviews['rating'][i]
matrix_all

<ipython-input-244-9d41e419b07c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_reviews['user_id_simple'] = pd.factorize(current_reviews.userId)[0]
<ipython-input-244-9d41e419b07c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_reviews['movie_id_simple'] = pd.factorize(current_reviews.movieId)[0]


array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [4. , 0. , 0. , ..., 0. , 0. , 0. ],
       [4.5, 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 3.5, 3.5, 0. ]])

In [222]:
current_reviews

,userId,movieId,rating,user_id_simple,movie_id_simple
0,1,1,4.0,0,0
1,5,1,4.0,1,0
2,7,1,4.5,2,0
3,15,1,2.5,3,0
4,17,1,4.5,4,0
...,...,...,...,...,...
100831,184,193581,4.0,609,9719
100832,184,193583,3.5,609,9720
100833,184,193585,3.5,609,9721
100834,184,193587,3.5,609,9722


In [245]:
for i in range(0,unique_users):
    user_id = current_reviews.loc[current_reviews.user_id_simple==i].userId.unique()[0]
    for j in range(0, unique_restaurants):
        if matrix_all[i][j] == 0:    
            star2=loaded_model.predict(i, j, r_ui=None, verbose=False)
            matrix_all[i][j] =  int(star2.est)
            #print(star2)


In [246]:
# llena los valores que si calificó el usuario con 0
for i in current_reviews.index:
    #print(current_reviews['userId'][i])
    matrix_all[current_reviews['user_id_simple'][i], current_reviews['movie_id_simple'][i]] = 0
matrix_all

array([[0., 3., 0., ..., 3., 3., 3.],
       [0., 3., 3., ..., 3., 3., 3.],
       [0., 3., 3., ..., 3., 3., 3.],
       ...,
       [3., 4., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 0., 0., 3.]])

## conectarse a mysql

In [247]:
!pip install mysql
!pip install mysql.connector
!pip install mysql-connector-python-rf
import mysql.connector
from mysql.connector import errorcode
#Configuración de la conexión a Mysql
try:
  cnx = mysql.connector.connect(user='user_taller3', password='taller3.', host='127.0.0.1', database='taller3')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

cursor = cnx.cursor()

In [ ]:
sql = "INSERT INTO `recomendations_movies` (`user_id`, `movie_id`, `recomendation_score`) VALUES (%s, %s, %s)"

for i in range(0,unique_users):
    user_id = current_reviews.loc[current_reviews.user_id_simple==i].userId.unique()[0]
    #print(user_id)
    for j in range(0, unique_restaurants):
        if matrix_all[i][j] >=4:    
            for k in current_reviews.index:
                if current_reviews['movie_id_simple'] [k]== j :
                    business_id = current_reviews['movieId'][k]
                    break
            #print(business_id)
            list_rec = recommendations(business_id)
            if len(list_rec) >0:
                for k in list_rec:
                    cursor.execute(sql, (str(user_id),str(k),int(matrix_all[i][j])))
            else:
                cursor.execute(sql, (str(user_id),str(business_id),int(matrix_all[i][j])))
            cnx.commit()